In [4]:
import openpyxl
import pandas as pd
from natsort import index_natsorted
import numpy as np

In [5]:
be = pd.read_excel("../raw/II_Inkunabeln+_test_Wendler.xlsx", sheet_name="II_Inkunabeln+", dtype="string")
ii = pd.read_csv("../abzug/ii.csv", dtype="string")

/home/wendler/.cache/pypoetry/virtualenvs/vrekon-e7A2tzLD-py3.10/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


# Versuch nur eine Spalte mit ja/nein Markierung einzubringen

Problem: die neuen IDNs kommen so nicht mit. Nützt also nichts.

In [5]:
# Merge enthält alle Sätze, die in meiner Liste und in der BE-Liste stehen
"2" in be.merge(ii, how="right", left_on=["AKZ", "IDN"], right_on=["akz", "idn"])["Lfd Nr."].values

True

In [12]:
merge_numbers = be.merge(ii, how="right", left_on=["AKZ", "IDN"], right_on=["akz", "idn"])["Lfd Nr."].values
def check_digi(nr: str):
    return nr in merge_numbers

In [13]:
gesamt = pd.DataFrame(be["Lfd Nr."])

In [14]:
gesamt["digi"] = gesamt["Lfd Nr."].apply(check_digi)

In [15]:
# gesamt enthält in spalte digi info, ob datensatz teil des projektes oder nicht.
gesamt

,Lfd Nr.,digi
0,1,True
1,2,True
2,3,True
3,4,True
4,5,True
...,...,...
731,707,False
732,708,False
733,709,False
734,710,False


# neuer Ansatz: merge

In [6]:
df = be.merge(ii, how="outer", left_on=["AKZ", "IDN"], right_on=["akz", "idn"])

# idn, akz und signatur werden aus abzugs-listen über die BE-Daten kopiert: dabei entsteht kein Datenverlust, weil diese Spalten identisch sein müssen, denn mit ihnen wurden die Daten gemerged
df.loc[df["IDN"].isna(), "IDN"] = df.idn
df.loc[df["AKZ"].isna(), "AKZ"] = df.akz
df.loc[df["Signatur"].isna(), "Signatur"] = df.signatur_a

# wenn der Datensatz in den Abzugs-Listen war, wird er mit True in der Spalte Digi gekennzeichnet, wenn nicht mit False; dabei bekommen auch Zeilen ein False, für die es noch keinen Datensatz gibt. Beim Durchgang im nächsten Monat wird das automatisch geändert, weil die dann neu angelegten Datensätze in der Abzugs-Liste erscheinen und damit auch hier eingefügt werden.
df.loc[df["idn"].notna(), "digi"] = True
df.loc[df["idn"].isna(), "digi"] = False
df.fillna('', inplace=True)
df = df.sort_values(by='Signatur', ascending=True, na_position='first', key=lambda X: np.argsort(index_natsorted(df["Signatur"])))

# die Spalten aus der Abzugs-Liste werden wieder entfernt, die Digi Spalte wird an den Anfang gesetzt
spaltenliste = list(df.columns)[:-22]
spaltenliste.insert(0,"digi")

df.to_excel("ii-merge-test.xlsx", index=False, columns=spaltenliste)

# Anleitung zur Bearbeitung der zusammengefügten Listen

Die Tabelle enthält ganz vorn die neue Spalte "Digi". Steht dort WAHR, wird das Werk digitalisiert, steht FALSCH, wird es nicht digitalisiert.

Zunächst muss überprüft werden, ob Frau Thomschke Daten in eine Zeile erfasst hat, in der eigentlich angebundene Werke oder sonstige untergeordnete Werke stehen. Diese müssen dann in die Zeile des übergeordneten Satzes verschoben werden, z. B. Qd.

Alle mit FALSCH markierten Zeilen, die angebundene Werke und keine Informationen von Frau Thomschke haben, können gelöscht werden.

Alle mit FALSCH markierte Zeilen, die keine IDN und keine AKZ enthalten, aber eine Signatur, wurden von BE "gefunden" für diese Bände muss ein Datensatz angelegt werden. Wenn das passiert ist, wird die IDN und AKZ in die Tabelle kopiert.







